In [1]:
import dask.dataframe as dd
import pandas as pd

In [9]:
data_dir = '../data/raw/'
df_sales = pd.read_csv(data_dir+'sales.csv', delimiter='\t')
df_cust = pd.read_csv(data_dir+'Customers.csv', delimiter='\t')
df_mail_train = pd.read_csv(data_dir+'Mailings1_2.csv')
df_demog = pd.read_csv(data_dir+'Demog.csv', delimiter='\t')
df_mail_test = pd.read_csv(data_dir+'Mailings3.txt',delimiter='\t')

In [12]:
df_sales.head(3)

,EVENTID,REFID,EVENT_DATE,AMOUNT
0,0,0,2006-12-21 12:00:00,17.907000
1,1,0,2006-12-25 12:00:00,17.400999
2,2,0,2007-01-26 12:00:00,13.277000


In [5]:
df_cust.head(3)

,ID,SEX,MARITAL_STATUS,GEOID,EDUCATIONNUM,OCCUPATION,DATA1,DATA2,DATA3,NOM1,NOM2,NOM3,age
0,0,Male,Never-married,61,7,Machine-op-inspct,9.276,3.592,8.253,l,p,o,62
1,1,Male,Married-civ-spouse,70,3,Handlers-cleaners,5.836,4.574,5.507,u,r,x,78
2,2,Male,Never-married,53,7,Machine-op-inspct,8.998,4.184,8.593,n,q,o,69


In [11]:
df_mail_train.head(3)

,REFID,KxIndex,REF_DATE,RESPONSE
0,0,1,2007-02-01 12:00:00,False
1,1,2,2007-02-01 12:00:00,False
2,2,3,2007-02-01 12:00:00,False


In [13]:
df_demog.head(3)

,GEOID,INHABITANTS_K,INCOME_K,A_VAR1,A_VAR2,A_VAR3,A_VAR4,A_VAR5,A_VAR6,A_VAR7,...,A_VAR9,A_VAR10,A_VAR11,A_VAR12,A_VAR13,A_VAR14,A_VAR15,A_VAR16,A_VAR17,A_VAR18
0,0,30.046,2631.468018,6.084,5.790,8.595,3.935,6.362,8.626,4.624,...,5.233,6.232,5.205,8.231,6.746,8.679,5.292,3.500,5.512,5.783
1,1,36.250,3012.745117,4.604,8.309,6.007,5.938,8.773,3.579,6.349,...,6.884,7.062,7.319,3.720,6.405,7.202,4.932,7.969,8.150,5.633
2,2,47.645,2192.410889,4.911,8.557,5.934,6.494,9.172,3.202,6.157,...,7.942,7.901,7.928,2.330,6.029,6.455,4.720,8.564,8.342,4.938


In [14]:
df_mail_test.head(3)

,REFID,REF_DATE,RESPONSE
0,60000,2007-07-01 12:00:00,False
1,60001,2007-07-01 12:00:00,False
2,60002,2007-07-01 12:00:00,False
